In [14]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import xarray as xr
from datetime import datetime, timedelta
import ee

# import geopandas as gpd
# import geemap
# import urllib.request
# from tqdm import tqdm
# import httplib2
# import os
# import socks

In [15]:
# from dask.distributed import Client, progress

# client = Client()  # set up local cluster on the machine

# client

In [16]:
ee.Initialize()

In [17]:
# Verificar que se trate del centro de CABA!

COORDs = [
 [-59.01657927, -34.36323361],
 [-59.01268172, -34.84847940],
 [-58.35274585, -34.84317871],
 [-58.35664341, -34.35793292]]

ROI = ee.Geometry.Polygon(COORDs)

In [18]:
# Get the latest available forecast

DATE_STARTs = [f"{datetime.strftime(datetime.now(), '%Y-%m-%d')}T{HH}:00" for HH in ["00", "06", "12", "18"]]

DATE_ENDs = [(datetime.strptime(D, '%Y-%m-%dT%H:%M') + timedelta(hours=6)).strftime('%Y-%m-%dT%H:%M') for D in DATE_STARTs]

In [19]:
# Dictionar of image collections
DICT_Cs = {}

for S, E, DESC in zip(DATE_STARTs, DATE_ENDs, ["T00", "T06", "T12", "T18"]):
    
    DICT_Cs[DESC] = ee.ImageCollection("NOAA/GFS0P25") \
                        .filterBounds(ROI) \
                        .filterDate(S, E) \
                        .filterMetadata("forecast_hours", "greater_than", 0)
DICT_Cs

{'T00': <ee.imagecollection.ImageCollection at 0x1d8f4876950>,
 'T06': <ee.imagecollection.ImageCollection at 0x1d8ff620890>,
 'T12': <ee.imagecollection.ImageCollection at 0x1d8ff6281d0>,
 'T18': <ee.imagecollection.ImageCollection at 0x1d8ff628810>}

In [20]:
# Almacena la cantidad de elementos en cada colección
SIZEs = []

for C in DICT_Cs.keys():
    
    SIZEs.append(DICT_Cs[C].size().getInfo())

# Get index of latest non zero value of list
IDX = -1

# for i in SIZEs:
for i in SIZEs:
    
    if i != 0:
        
        IDX += 1
    
    else:
        
        pass

# Generar la clave de la colección de interés
if IDX == 0:
    
    N="T00"

elif IDX == 1:
    
    N="T06"
    
elif IDX == 2:
    
    N="T12"
    
else:

    N="T18"

# print(N)

In [21]:
# Keep last forecast from dictionary
C_01 = DICT_Cs[N] # Devuelve 208 elementos (120 imágenes corresponden a los primeros 5 días, las 88 imágenes restantes corresponden a los últimos 11 días)

C_01

In [22]:
# Projection
PROJ = C_01.first().select(0).projection()

# Original scale
OS = C_01.first().projection().nominalScale().getInfo()

# print(f"Original scale: {OS:.1f} m")

In [23]:
# Convertir en un DS de xarray
DS_01 = xr.open_dataset(C_01, \
                     engine='ee',\
                     projection=PROJ,\
                     geometry=ROI\
                     )
DS_01

<xarray.Dataset>
Dimensions:                               (time: 208, lon: 3, lat: 2)
Coordinates:
  * time                                  (time) datetime64[ns] 2024-08-24 .....
  * lon                                   (lon) float32 -58.89 -58.64 -58.39
  * lat                                   (lat) float32 -34.66 -34.91
Data variables:
    temperature_2m_above_ground           (time, lon, lat) float64 ...
    specific_humidity_2m_above_ground     (time, lon, lat) float64 ...
    relative_humidity_2m_above_ground     (time, lon, lat) float64 ...
    u_component_of_wind_10m_above_ground  (time, lon, lat) float64 ...
    v_component_of_wind_10m_above_ground  (time, lon, lat) float64 ...
    total_precipitation_surface           (time, lon, lat) float64 ...
    precipitable_water_entire_atmosphere  (time, lon, lat) float64 ...
    total_cloud_cover_entire_atmosphere   (time, lon, lat) float64 ...
    downward_shortwave_radiation_flux     (time, lon, lat) float64 ...
Attributes: (12/19)
    date_range:               [1435708800000, 1647993600000]
    description:              <p>The Global Forecast System (GFS) is a weathe...
    keywords:                 ['climate', 'cloud', 'emc', 'flux', 'forecast',...
    period:                   0
    product_tags:             ['temperature', 'humidity', 'wind', 'radiation'...
    provider:                 NOAA/NCEP/EMC
    ...                       ...
    visualization_0_bands:    temperature_2m_above_ground
    visualization_0_max:      35.0
    visualization_0_min:      -40.0
    visualization_0_name:     Temperature Above Ground
    visualization_0_palette:  blue,purple,cyan,green,yellow,red
    crs:                      GEOGCS["unknown", \n  DATUM["unnamed", \n    SP...

In [24]:
# Filtrar los primeros 120 registros

DS_01 = DS_01.isel(time=slice(0, 120))

DS_01

<xarray.Dataset>
Dimensions:                               (time: 120, lon: 3, lat: 2)
Coordinates:
  * time                                  (time) datetime64[ns] 2024-08-24 .....
  * lon                                   (lon) float32 -58.89 -58.64 -58.39
  * lat                                   (lat) float32 -34.66 -34.91
Data variables:
    temperature_2m_above_ground           (time, lon, lat) float64 ...
    specific_humidity_2m_above_ground     (time, lon, lat) float64 ...
    relative_humidity_2m_above_ground     (time, lon, lat) float64 ...
    u_component_of_wind_10m_above_ground  (time, lon, lat) float64 ...
    v_component_of_wind_10m_above_ground  (time, lon, lat) float64 ...
    total_precipitation_surface           (time, lon, lat) float64 ...
    precipitable_water_entire_atmosphere  (time, lon, lat) float64 ...
    total_cloud_cover_entire_atmosphere   (time, lon, lat) float64 ...
    downward_shortwave_radiation_flux     (time, lon, lat) float64 ...
Attributes: (12/19)
    date_range:               [1435708800000, 1647993600000]
    description:              <p>The Global Forecast System (GFS) is a weathe...
    keywords:                 ['climate', 'cloud', 'emc', 'flux', 'forecast',...
    period:                   0
    product_tags:             ['temperature', 'humidity', 'wind', 'radiation'...
    provider:                 NOAA/NCEP/EMC
    ...                       ...
    visualization_0_bands:    temperature_2m_above_ground
    visualization_0_max:      35.0
    visualization_0_min:      -40.0
    visualization_0_name:     Temperature Above Ground
    visualization_0_palette:  blue,purple,cyan,green,yellow,red
    crs:                      GEOGCS["unknown", \n  DATUM["unnamed", \n    SP...

In [25]:
FyH = pd.date_range(start=DATE_STARTs[IDX], freq="1H", periods=120+1)[1:]

DS_01 = DS_01.assign_coords(FyH=("time", FyH))

DS_01 = DS_01.swap_dims({"time" : "FyH"})
                            
DS_01 = DS_01.drop_vars("time")
    
DS_01

<xarray.Dataset>
Dimensions:                               (FyH: 120, lon: 3, lat: 2)
Coordinates:
  * lon                                   (lon) float32 -58.89 -58.64 -58.39
  * lat                                   (lat) float32 -34.66 -34.91
  * FyH                                   (FyH) datetime64[ns] 2024-08-24T01:...
Data variables:
    temperature_2m_above_ground           (FyH, lon, lat) float64 ...
    specific_humidity_2m_above_ground     (FyH, lon, lat) float64 ...
    relative_humidity_2m_above_ground     (FyH, lon, lat) float64 ...
    u_component_of_wind_10m_above_ground  (FyH, lon, lat) float64 ...
    v_component_of_wind_10m_above_ground  (FyH, lon, lat) float64 ...
    total_precipitation_surface           (FyH, lon, lat) float64 ...
    precipitable_water_entire_atmosphere  (FyH, lon, lat) float64 ...
    total_cloud_cover_entire_atmosphere   (FyH, lon, lat) float64 ...
    downward_shortwave_radiation_flux     (FyH, lon, lat) float64 ...
Attributes: (12/19)
    date_range:               [1435708800000, 1647993600000]
    description:              <p>The Global Forecast System (GFS) is a weathe...
    keywords:                 ['climate', 'cloud', 'emc', 'flux', 'forecast',...
    period:                   0
    product_tags:             ['temperature', 'humidity', 'wind', 'radiation'...
    provider:                 NOAA/NCEP/EMC
    ...                       ...
    visualization_0_bands:    temperature_2m_above_ground
    visualization_0_max:      35.0
    visualization_0_min:      -40.0
    visualization_0_name:     Temperature Above Ground
    visualization_0_palette:  blue,purple,cyan,green,yellow,red
    crs:                      GEOGCS["unknown", \n  DATUM["unnamed", \n    SP...

In [26]:
DS_01["FH"] = xr.DataArray(np.arange(1, 121), dims="FyH")

DS_01["H"] = DS_01["FH"] % 6

# Pulsos de ppt discreta
DS_01["PPT_D"] = DS_01["total_precipitation_surface"].diff(dim="FyH")

DS_01["PPT_D"] = DS_01["PPT_D"].where(DS_01["H"].shift(FyH=1) != 0, DS_01["total_precipitation_surface"])

DS_01

<xarray.Dataset>
Dimensions:                               (lon: 3, lat: 2, FyH: 120)
Coordinates:
  * lon                                   (lon) float32 -58.89 -58.64 -58.39
  * lat                                   (lat) float32 -34.66 -34.91
  * FyH                                   (FyH) datetime64[ns] 2024-08-24T01:...
Data variables:
    temperature_2m_above_ground           (FyH, lon, lat) float64 ...
    specific_humidity_2m_above_ground     (FyH, lon, lat) float64 ...
    relative_humidity_2m_above_ground     (FyH, lon, lat) float64 ...
    u_component_of_wind_10m_above_ground  (FyH, lon, lat) float64 ...
    v_component_of_wind_10m_above_ground  (FyH, lon, lat) float64 ...
    total_precipitation_surface           (FyH, lon, lat) float64 0.0 ... 0.0
    precipitable_water_entire_atmosphere  (FyH, lon, lat) float64 ...
    total_cloud_cover_entire_atmosphere   (FyH, lon, lat) float64 ...
    downward_shortwave_radiation_flux     (FyH, lon, lat) float64 ...
    FH                                    (FyH) int32 1 2 3 4 ... 118 119 120
    H                                     (FyH) int32 1 2 3 4 5 0 ... 2 3 4 5 0
    PPT_D                                 (FyH, lon, lat) float64 nan ... 0.0
Attributes: (12/19)
    date_range:               [1435708800000, 1647993600000]
    description:              <p>The Global Forecast System (GFS) is a weathe...
    keywords:                 ['climate', 'cloud', 'emc', 'flux', 'forecast',...
    period:                   0
    product_tags:             ['temperature', 'humidity', 'wind', 'radiation'...
    provider:                 NOAA/NCEP/EMC
    ...                       ...
    visualization_0_bands:    temperature_2m_above_ground
    visualization_0_max:      35.0
    visualization_0_min:      -40.0
    visualization_0_name:     Temperature Above Ground
    visualization_0_palette:  blue,purple,cyan,green,yellow,red
    crs:                      GEOGCS["unknown", \n  DATUM["unnamed", \n    SP...

In [27]:
DF_01 = DS_01.mean(dim=("lon", "lat")).to_dataframe()

DF_01["CUMSUM"] = DF_01["PPT_D"].cumsum()

DF_01.index = DF_01.index - pd.offsets.Hour(3)

DF_01

,temperature_2m_above_ground,specific_humidity_2m_above_ground,relative_humidity_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,total_precipitation_surface,precipitable_water_entire_atmosphere,total_cloud_cover_entire_atmosphere,downward_shortwave_radiation_flux,FH,H,PPT_D
FyH,,,,,,,,,,,,
2024-08-24 01:00:00,6.769179,0.004248,70.650002,1.894537,1.481822,0.0,8.436734,54.450001,0.000000,1,1,NaN
2024-08-24 02:00:00,6.403375,0.004157,70.833333,1.873191,1.397562,0.0,8.627465,87.250001,0.000000,2,2,0.0
2024-08-24 03:00:00,6.186711,0.004101,70.983335,1.652448,1.579007,0.0,8.821523,95.816667,0.000000,3,3,0.0
2024-08-24 04:00:00,5.835881,0.004071,72.283334,1.686077,1.461139,0.0,9.221529,74.850000,0.000000,4,4,0.0
2024-08-24 05:00:00,5.470951,0.004037,73.566668,1.621567,1.364299,0.0,9.439989,11.183334,0.000000,5,5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-28 20:00:00,15.182937,0.006139,58.233335,-4.449800,1.041003,0.0,13.218848,100.000000,437.436656,116,2,0.0
2024-08-28 21:00:00,13.566670,0.006132,64.533334,-4.022383,1.649044,0.0,14.152507,100.000000,333.176656,117,3,0.0
2024-08-28 22:00:00,12.250162,0.005883,67.550001,-3.956977,2.003443,0.0,14.757254,100.000000,252.597346,118,4,0.0
